## sample1
### 回路
![サンプル1の回路](circuit_sample1.png)
### プログラム

In [ ]:
import requests
import json
import wiringpi
import time
from watson_developer_cloud import TextToSpeechV1
import pygame.mixer

def get_forecast():
    url = 'http://weather.livedoor.com/forecast/webservice/json/v1?city=130010'
    data = requests.get(url).json()

    location = data['location']['city']
    forecasts = data['forecasts']
    tomorrow_forecast = forecasts[1]
    tomorrow_weather = tomorrow_forecast['telop']

    forecast = '明日の{0}の天気は{1}です'.format(location, tomorrow_weather)

    return forecast

def text2speech(text, filename):
    api_key = ''
    url = 'https://stream.watsonplatform.net/text-to-speech/api'

    text_to_speech = TextToSpeechV1(iam_apikey=api_key, url=url)
    r = text_to_speech.synthesize(text, 'audio/mp3', 'ja-JP_EmiVoice').get_result().content

    with open(filename, 'wb') as audio_file:
        audio_file.write(r)

def play(filename):
    pygame.mixer.init()
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()
    time.sleep(5)
    pygame.mixer.music.stop()
    pygame.mixer.quit()

def getDistance(trig_pin, echo_pin):
    ### trigger
    wiringpi.digitalWrite(trig_pin, wiringpi.HIGH)
    time.sleep(0.00001)  # [sec]
    wiringpi.digitalWrite(trig_pin, wiringpi.LOW)

    ### response time
    while wiringpi.digitalRead(echo_pin) == 0:
        time_begin = time.time()
    
    while wiringpi.digitalRead(echo_pin) == 1:
        time_end = time.time()

    t = time_end - time_begin
    
    ### calculate distance
    distance = t * 17000

    return distance

def save_forecast_audio(forecast_filename):
    forecast = get_forecast()  # forecastは英語で「予報」という意味
    print(forecast)

    text2speech(forecast, forecast_filename)

def main():
    trig_pin = 17
    echo_pin = 27

    wiringpi.wiringPiSetupGpio()
    wiringpi.pinMode(trig_pin, wiringpi.OUTPUT)
    wiringpi.pinMode(echo_pin, wiringpi.INPUT)

    wiringpi.digitalWrite(trig_pin, wiringpi.LOW)
    forecast_filename = 'forecast.mp3'

    while True:
        distance = getDistance(trig_pin, echo_pin)
        print(distance)

        time.sleep(0.5)  # [sec]

        if distance < 15:
            save_forecast_audio(forecast_filename)
            time.sleep(1)
            play(forecast_filename)
        if distance < 8:
            break

main()

## sample2
### 回路
![サンプル2の回路](circuit_sample2.png)

### プログラム

In [ ]:
import wiringpi
import time

def getDistance(trig_pin, echo_pin):
    ### trigger
    wiringpi.digitalWrite(trig_pin, wiringpi.HIGH)
    time.sleep(0.00001)  # [sec]
    wiringpi.digitalWrite(trig_pin, wiringpi.LOW)

    ### response time
    while wiringpi.digitalRead(echo_pin) == 0:
        time_begin = time.time()
    
    while wiringpi.digitalRead(echo_pin) == 1:
        time_end = time.time()

    t = time_end - time_begin
    
    ### calculate distance
    distance = t * 17000

    return distance

def main():
    trig_pin = 17
    echo_pin = 27

    wiringpi.wiringPiSetupGpio()
    wiringpi.pinMode(trig_pin, wiringpi.OUTPUT)
    wiringpi.pinMode(echo_pin, wiringpi.INPUT)

    wiringpi.digitalWrite(trig_pin, wiringpi.LOW)
    count = 0
    body_up = False

    print('腹筋スタート！')

    start_time = time.time()
    current_time = time.time()

    while (current_time - start_time) < 10:  # 10秒間カウント
        distance = getDistance(trig_pin, echo_pin)

        time.sleep(0.5)  # [sec]

        if distance < 8 and body_up == False:
            count += 1
            body_up = True
            print('count: {}'.format(count))
        if distance > 20 and body_up == True:
            body_up = False

        current_time = time.time()

    print('腹筋回数：{}回'.format(count))
    print('お疲れ様でした！')

main()

## sample3
### 回路
![サンプル3の回路](circuit_sample3.png)

### プログラム

In [ ]:
import wiringpi
import time
import picamera
from watson_developer_cloud import VisualRecognitionV3
import json

def getDistance(trig_pin, echo_pin):
    ### trigger
    wiringpi.digitalWrite(trig_pin, wiringpi.HIGH)
    time.sleep(0.00001)  # [sec]
    wiringpi.digitalWrite(trig_pin, wiringpi.LOW)

    ### response time
    while wiringpi.digitalRead(echo_pin) == 0:
        time_begin = time.time()
    
    while wiringpi.digitalRead(echo_pin) == 1:
        time_end = time.time()

    t = time_end - time_begin
    
    ### calculate distance
    distance = t * 17000

    return distance

def estimate_age(filename):
    version = '2018-03-19'
    api_key = ''

    recognizer = VisualRecognitionV3(version=version, iam_apikey=api_key)
    with open (filename, 'rb') as image:
        results = recognizer.detect_faces(image).get_result()

    r = results['images'][0]['faces']

    age_min = float(r[0]['age']['min'])
    age_max = float(r[0]['age']['max'])
    age = int((age_max + age_min) / 2.0 + 0.5)

    return age

def main():
    trig_pin = 17
    echo_pin = 27

    wiringpi.wiringPiSetupGpio()
    wiringpi.pinMode(trig_pin, wiringpi.OUTPUT)
    wiringpi.pinMode(echo_pin, wiringpi.INPUT)

    wiringpi.digitalWrite(trig_pin, wiringpi.LOW)

    cam = picamera.PiCamera()
    cam.resolution = (640, 480)
    cam.start_preview()
    filename = 'my_picture.png'

    while True:
        distance = getDistance(trig_pin, echo_pin)
        print(distance)

        time.sleep(0.5)  # [sec]

        if distance < 15:
            print('capture')
            cam.capture(filename)
            result = estimate_age(filename)
            print('推定年齢：{}才'.format(result))
        if distance < 8:
            break

    cam.stop_preview()

main()